In [ ]:
# import necessary libraries
import pandas as pd

# read the necessary dataset
df = pd.read_csv("../input/survey_results_public.csv")

# take a look at the dataset
df.head()

In [ ]:
# get a first overview of the relevant columns

# size of dataset
print(df.shape)

# missing values for 'CompanySize' and 'HomeRemote'
print(np.sum(df['CompanySize'].isnull() == 1))
print(np.sum(df['HomeRemote'].isnull() == 1))

# proportion of missing values for 'CompanySize' and 'HomeRemote'
print(df['CompanySize'].isnull().mean())
print(df['HomeRemote'].isnull().mean())

In [ ]:
# take a look at the answers for 'CompanySize'
df['CompanySize'].value_counts(dropna=False).reset_index()

In [ ]:
# for the purpose of this task, ignore all rows that do not give a specific answer for 'CompanySize'
df_work = df.dropna(subset=['CompanySize'], axis=0)
df_work = df_work[df_work['CompanySize'] != "I don't know"]
df_work = df_work[df_work['CompanySize'] != "I prefer not to answer"]

df_work.shape

In [ ]:
# take a look at the answers for 'HomeRemote'
df['HomeRemote'].value_counts(dropna=False).reset_index()

In [ ]:
# for the purpose of this task, ignore all rows that do not give a specific answer for 'HomeRemote'

df_work = df_work.dropna(subset=['HomeRemote'], axis=0)
df_work = df_work[df_work['HomeRemote'] != "It's complicated"]
df_work.shape

In [ ]:
# take a look at the answers for 'HomeRemote' grouped by 'CompanySize'
df_work.groupby(['CompanySize', 'HomeRemote']).count()['Respondent']

In [ ]:
# define a function to divide all the answers for 'HomeRemote' into either "Yes" or "No"

def f(row):
    if row == "Never":
        return 'No'
    else:
        return 'Yes'
    
# apply the above function
df_work['HomeRemote2'] = df_work['HomeRemote'].apply(f)

In [ ]:
# take a look at the data
df_work.groupby(['CompanySize', 'HomeRemote2']).count()['Respondent']

In [ ]:
# create individual dataframes for the results
df_result_yes = df_work[df_work['HomeRemote2'] == "Yes"].groupby(['CompanySize', 'HomeRemote2']).count()['Respondent'].reset_index()
df_result_no = df_work[df_work['HomeRemote2'] == "No"].groupby(['CompanySize', 'HomeRemote2']).count()['Respondent'].reset_index()

# add a sorting column
df_result_yes['sort'] = [6, 2, 8, 4, 3, 7, 5, 1]
df_result_no['sort'] = [6, 2, 8, 4, 3, 7, 5, 1]

# print the results
print(df_result_yes.sort_values(by='sort', ascending=True))
print(df_result_no.sort_values(by='sort', ascending=True))

In [ ]:
# combine data into one dataset
df_result = df_result_yes
df_result.rename(columns={'Respondent': 'Yes'}, inplace=True)
df_result['No'] = df_result_no['Respondent']

# add percentages
df_result['Yes_Percent'] = df_result['Yes']/(df_result['No'] + df_result['Yes'])*100
df_result['No_Percent'] = df_result['No']/(df_result['No'] + df_result['Yes'])*100

# print results
df_final = df_result[['sort', 'CompanySize', 'Yes', 'No', 'Yes_Percent', 'No_Percent']].sort_values(by='sort', ascending=True)
df_final.head()

In [ ]:
# visualize results in a bar chart
df_final.plot('CompanySize', 'Yes_Percent', kind="bar", figsize=(15,5), legend=False, color="blue");
plt.title('Remote work based on Company Size', fontsize=15);